In [11]:
import torch
import torch.nn.functional as F

In [2]:
logits = torch.randn(20, 32)
logits

tensor([[-8.3289e-01, -3.7532e-01, -1.8049e+00, -1.1873e+00, -2.5143e+00,
         -4.3779e-01, -1.5243e-01, -1.4132e+00, -1.3822e+00,  1.5159e+00,
         -5.8740e-01, -1.1610e+00,  1.3189e+00,  6.2140e-01,  3.1834e-01,
          1.1852e+00, -1.2456e+00, -1.2198e+00, -6.8095e-01, -9.3521e-01,
          5.4539e-01,  1.8948e+00,  1.1441e+00,  1.0342e+00, -1.6833e-02,
         -5.4095e-02,  7.5545e-01,  5.4895e-01,  6.6276e-01,  3.5680e-01,
         -1.0821e+00,  3.3272e-01],
        [-2.1817e+00, -4.9041e-01, -6.1976e-01, -5.7327e-01, -7.3050e-01,
         -1.1674e+00,  1.5037e+00, -1.5489e+00, -5.5206e-01, -1.6347e+00,
         -8.2686e-01, -1.0037e+00, -4.3762e-01,  7.7285e-01, -2.3297e+00,
          5.0035e-01,  1.8790e+00, -4.8468e-01,  8.6224e-01, -3.5678e-01,
          1.7236e+00,  4.6389e-04, -8.8696e-01, -1.9520e+00, -1.6304e+00,
         -2.3636e+00, -1.5101e+00, -6.6748e-01, -4.4035e-01, -6.9837e-01,
          3.1021e-01,  9.4109e-01],
        [-9.4109e-01, -6.0933e-01, -1.36

In [4]:
# Sample soft categorical using reparametrization trick:
F.gumbel_softmax(logits, tau=1, hard=False)
# Sample hard categorical using "Straight-through" trick:
F.gumbel_softmax(logits, tau=1, hard=True)

tensor([[9.3276e-03, 4.1848e-03, 1.7100e-03, 1.4884e-03, 3.8685e-04, 4.9372e-03,
         1.3238e-02, 3.4188e-03, 8.9195e-04, 3.7121e-02, 1.4080e-02, 6.2456e-03,
         2.8920e-02, 5.6853e-02, 3.3346e-03, 9.7475e-02, 3.2392e-03, 8.5327e-03,
         6.6554e-03, 1.3953e-02, 2.0443e-02, 1.6846e-01, 2.2106e-01, 1.0588e-02,
         2.3714e-02, 1.8931e-02, 1.9513e-02, 2.8238e-02, 1.7648e-02, 1.0062e-01,
         9.2759e-03, 4.5518e-02],
        [2.3446e-03, 1.0397e-02, 9.5342e-02, 1.9514e-03, 1.5281e-03, 5.5212e-04,
         4.7910e-02, 7.3021e-04, 2.9806e-03, 1.2263e-03, 2.3316e-03, 3.9776e-03,
         5.1823e-03, 2.0586e-02, 6.4888e-04, 4.6690e-03, 4.0825e-02, 9.1261e-03,
         3.8768e-02, 4.2723e-03, 7.7592e-02, 1.3074e-02, 1.2576e-03, 2.5044e-03,
         4.6275e-04, 1.3893e-03, 6.3583e-04, 2.0254e-03, 4.6681e-03, 3.8385e-02,
         3.8080e-01, 1.8185e-01],
        [7.2437e-03, 1.6250e-03, 1.4832e-03, 3.9936e-01, 2.0402e-02, 3.1000e-02,
         1.1732e-02, 2.7012e-02, 9.9260e-

In [29]:
logits = torch.randn(4, requires_grad=True)
print("logits:", logits)
x = F.gumbel_softmax(logits, tau=1, hard=False)
print("x:", x)

ground_truth = torch.tensor([1.0, 1.0, 1.0, 1.0])
loss = torch.sum((x - ground_truth) ** 2)
# loss.backward(gradient = torch.tensor([1., 1., 1., 1.]))
loss.backward()

# print the gradient using .grad
print("grad:", logits.grad)

logits: tensor([ 1.1916, -0.8708, -0.2815,  1.2108], requires_grad=True)
x: tensor([0.6098, 0.0123, 0.0945, 0.2834], grad_fn=<SoftmaxBackward0>)
grad: tensor([ 0.1811, -0.0110, -0.0693, -0.1008])


In [4]:
import torch.nn as nn

# logits = torch.randn(4, requires_grad=True)

ground_truth = torch.tensor([1.0, 0, 0, 0])
loss_fn = nn.MSELoss()
kl_loss = nn.KLDivLoss()

In [5]:
class gumbel_m(nn.Module):
    def __init__(self, temp):
        super().__init__()
        self.logits = nn.Parameter(torch.randn(4))
        self.temp = temp

    def forward(self):
        x = F.gumbel_softmax(self.logits, tau=self.temp, hard=False)
        return x

In [4]:
model = gumbel_m()
print("initial distribution")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer = torch.optim.Adam(model.parameters())

initial distribution
logits tensor([-0.9917,  0.9721,  0.2368,  0.0959])


In [5]:
# Using MAE loss, distribution converges to target
for epoch in range(10001):
    optimizer.zero_grad()
    out = model()
    if epoch % 1000 == 0:
        print(out)
    # loss = kl_loss(out, ground_truth)
    loss = loss_fn(out, ground_truth)
    # print(loss)
    loss.backward()
    optimizer.step()

tensor([0.0455, 0.1650, 0.4725, 0.3171], grad_fn=<SoftmaxBackward0>)
tensor([0.0056, 0.1103, 0.8186, 0.0655], grad_fn=<SoftmaxBackward0>)
tensor([0.5409, 0.3669, 0.0435, 0.0487], grad_fn=<SoftmaxBackward0>)
tensor([0.9591, 0.0083, 0.0061, 0.0265], grad_fn=<SoftmaxBackward0>)
tensor([0.8960, 0.0160, 0.0122, 0.0759], grad_fn=<SoftmaxBackward0>)
tensor([0.8264, 0.1411, 0.0104, 0.0221], grad_fn=<SoftmaxBackward0>)
tensor([0.8719, 0.0919, 0.0037, 0.0325], grad_fn=<SoftmaxBackward0>)
tensor([0.9950, 0.0021, 0.0011, 0.0019], grad_fn=<SoftmaxBackward0>)
tensor([9.9486e-01, 7.5520e-04, 2.1441e-03, 2.2442e-03],
       grad_fn=<SoftmaxBackward0>)
tensor([0.9627, 0.0147, 0.0207, 0.0019], grad_fn=<SoftmaxBackward0>)
tensor([9.9516e-01, 3.2243e-04, 3.0928e-03, 1.4236e-03],
       grad_fn=<SoftmaxBackward0>)


In [7]:
model2 = gumbel_m()
print("initial distribution")
for name, param in model2.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer2 = torch.optim.Adam(model2.parameters())

initial distribution
logits tensor([-2.1847,  0.4396, -1.3671,  0.9186])


In [8]:
# Using KL Divergence loss, distribution converges to target
for epoch in range(10001):
    optimizer2.zero_grad()
    out = model2()
    if epoch % 1000 == 0:
        print(out)
    loss = kl_loss(out, ground_truth)
    # loss = loss_fn(out, ground_truth)
    # print(loss)
    loss.backward()
    optimizer2.step()

tensor([0.6275, 0.1467, 0.0493, 0.1765], grad_fn=<SoftmaxBackward0>)


/data/users1/dkim195/miniconda3/envs/env/lib/python3.11/site-packages/torch/nn/functional.py:2994: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor([0.0644, 0.1758, 0.3054, 0.4544], grad_fn=<SoftmaxBackward0>)
tensor([0.1309, 0.0326, 0.0032, 0.8333], grad_fn=<SoftmaxBackward0>)
tensor([9.9336e-01, 4.6259e-04, 8.4621e-04, 5.3346e-03],
       grad_fn=<SoftmaxBackward0>)
tensor([0.8771, 0.1093, 0.0029, 0.0107], grad_fn=<SoftmaxBackward0>)
tensor([9.9648e-01, 5.8527e-04, 2.6101e-03, 3.2602e-04],
       grad_fn=<SoftmaxBackward0>)
tensor([0.9432, 0.0389, 0.0031, 0.0148], grad_fn=<SoftmaxBackward0>)
tensor([9.9204e-01, 1.0175e-03, 1.4226e-04, 6.8028e-03],
       grad_fn=<SoftmaxBackward0>)
tensor([9.8124e-01, 9.5598e-04, 6.4678e-04, 1.7162e-02],
       grad_fn=<SoftmaxBackward0>)
tensor([9.9915e-01, 3.7352e-04, 5.5028e-05, 4.1997e-04],
       grad_fn=<SoftmaxBackward0>)
tensor([9.9437e-01, 2.8967e-04, 7.6745e-05, 5.2586e-03],
       grad_fn=<SoftmaxBackward0>)


In [6]:
# Testing high temp
# gradients pass back so converges okay

model3 = gumbel_m(temp=10)
print("initial distribution")
for name, param in model3.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer3 = torch.optim.Adam(model3.parameters())

# Using MLE loss, distribution converges to target
for epoch in range(10001):
    optimizer3.zero_grad()
    out = model3()
    if epoch % 1000 == 0:
        print(out)
    # loss = kl_loss(out, ground_truth)
    loss = loss_fn(out, ground_truth)
    # print(loss)
    loss.backward()
    optimizer3.step()

initial distribution
logits tensor([ 2.1702, -0.0610, -0.4752,  0.3097])
tensor([0.3114, 0.2445, 0.1994, 0.2448], grad_fn=<SoftmaxBackward0>)
tensor([0.3733, 0.1974, 0.2169, 0.2124], grad_fn=<SoftmaxBackward0>)
tensor([0.4204, 0.2068, 0.1887, 0.1842], grad_fn=<SoftmaxBackward0>)
tensor([0.4094, 0.1787, 0.2018, 0.2101], grad_fn=<SoftmaxBackward0>)
tensor([0.4838, 0.2018, 0.1484, 0.1661], grad_fn=<SoftmaxBackward0>)
tensor([0.5099, 0.1537, 0.1714, 0.1649], grad_fn=<SoftmaxBackward0>)
tensor([0.5613, 0.1574, 0.1481, 0.1332], grad_fn=<SoftmaxBackward0>)
tensor([0.5678, 0.1470, 0.1446, 0.1406], grad_fn=<SoftmaxBackward0>)
tensor([0.6395, 0.1088, 0.1241, 0.1276], grad_fn=<SoftmaxBackward0>)
tensor([0.6479, 0.1155, 0.0983, 0.1383], grad_fn=<SoftmaxBackward0>)
tensor([0.7288, 0.0898, 0.0912, 0.0902], grad_fn=<SoftmaxBackward0>)


In [7]:
# Testing low temp
# gradients aren't the best so convergence path is weird

model4 = gumbel_m(temp=0.01)
print("initial distribution")
for name, param in model4.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer4 = torch.optim.Adam(model4.parameters())

# Using MLE loss, distribution converges to target
for epoch in range(10001):
    optimizer4.zero_grad()
    out = model4()
    if epoch % 1000 == 0:
        print(out)
    # loss = kl_loss(out, ground_truth)
    loss = loss_fn(out, ground_truth)
    # print(loss)
    loss.backward()
    optimizer4.step()

initial distribution
logits tensor([ 0.7105,  1.7994, -1.0446, -1.4619])
tensor([1.0000e+00, 1.6669e-32, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([1.0000e+00, 7.3876e-10, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([0., 1., 0., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 1., 0., 0.], grad_fn=<SoftmaxBackward0>)
tensor([5.6302e-19, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([1.7870e-04, 9.9982e-01, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([7.3521e-14, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([1.7495e-11, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([1.0000e+00, 0.0000e+00, 4.7773e-21, 0.0000e+00],
       grad_fn=<SoftmaxBackward0>)
tensor([1., 0., 0., 0.], grad_fn=<SoftmaxBackward0>)


In [10]:
# Testing very low temp
# no gradients so does not converge

model5 = gumbel_m(temp=1e-10)
print("initial distribution")
for name, param in model5.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer5 = torch.optim.Adam(model5.parameters())

# Using MLE loss, distribution converges to target
for epoch in range(10001):
    optimizer5.zero_grad()
    out = model5()
    if epoch % 1000 == 0:
        print(out)
    # loss = kl_loss(out, ground_truth)
    loss = loss_fn(out, ground_truth)
    # print(loss)
    loss.backward()
    optimizer5.step()

initial distribution
logits tensor([-1.0412, -1.0363,  0.3127,  1.0133])
tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 1., 0., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 1., 0.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward0>)
tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward0>)


In [43]:
temp1 = []
for _ in range(10000):
    temp1.append(F.gumbel_softmax(torch.Tensor([0.3, 0.3, 0.4]), tau=0.01, hard=False))
np.mean(np.array(temp1), axis=0)

array([0.32190356, 0.31582075, 0.36227572], dtype=float32)

In [36]:
F.softmax(torch.Tensor([0.3, 0.3, 0.4]))

/tmp/ipykernel_3439453/156050755.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.Tensor([0.3, 0.3, 0.4]))


tensor([0.3220, 0.3220, 0.3559])

In [73]:
class mask_learning(nn.Module):
    def __init__(self, temp):
        super().__init__()
        self.logits = nn.Parameter(torch.randn(3))
        self.temp = temp
        self.lin1 = nn.Linear(3, 2)

    def forward(self, x):
        mask = F.gumbel_softmax(self.logits, tau=self.temp, hard=True)
        y_pred = self.lin1(mask * x)
        return y_pred

    def softmax(self):
        return F.softmax(self.logits)

In [74]:
x = torch.randn(1000, 3)
y = x[:, 1:]

In [75]:
model = mask_learning(temp=1)
print("initial distribution")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
optimizer = torch.optim.Adam(model.parameters())

initial distribution
logits tensor([ 0.1570, -0.2804,  1.9538])
lin1.weight tensor([[ 0.2386, -0.1962, -0.1054],
        [-0.0572, -0.0825, -0.3542]])
lin1.bias tensor([-0.4225,  0.4419])


In [76]:
# Using MAE loss, distribution converges to target
for epoch in range(10001):
    optimizer.zero_grad()
    out = model(x)
    # loss = kl_loss(out, ground_truth)
    loss = loss_fn(out, y)
    if epoch % 1000 == 0:
        print(model.softmax())
        print(float(loss))
    loss.backward()
    optimizer.step()

/tmp/ipykernel_3439453/2159838833.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.logits)


tensor([0.1303, 0.0841, 0.7856], grad_fn=<SoftmaxBackward0>)
1.617917776107788
tensor([0.0991, 0.0457, 0.8551], grad_fn=<SoftmaxBackward0>)
0.6916490197181702
tensor([0.0280, 0.0176, 0.9545], grad_fn=<SoftmaxBackward0>)
0.5084733366966248
tensor([0.0193, 0.0189, 0.9618], grad_fn=<SoftmaxBackward0>)
0.4982832074165344
tensor([0.0182, 0.0281, 0.9537], grad_fn=<SoftmaxBackward0>)
0.4982263445854187
tensor([0.0200, 0.0675, 0.9125], grad_fn=<SoftmaxBackward0>)
0.4982301592826843
tensor([0.0197, 0.1803, 0.8001], grad_fn=<SoftmaxBackward0>)
0.5179229974746704
tensor([0.0164, 0.3316, 0.6520], grad_fn=<SoftmaxBackward0>)
0.4983106255531311
tensor([0.0122, 0.4432, 0.5446], grad_fn=<SoftmaxBackward0>)
0.5094834566116333
tensor([0.0093, 0.4584, 0.5323], grad_fn=<SoftmaxBackward0>)
1.011027216911316
tensor([0.0071, 0.4912, 0.5017], grad_fn=<SoftmaxBackward0>)
0.49832069873809814
